<center>
     <span style = "font-family:Arial; font-size:20pt; font-weight:bold;color:purple;">Library Management and Book Recommender System (LMBRS) </span>
    <br/><br/>
    <span style = "font-family:Arial; font-size:20pt; font-weight:bold;">Data Processing & DB Creation</span>
    <br/><br/><br/>
    <span style = "font-family:Arial; font-size:12pt; font-weight:bold;text-decoration:underline;line-height: 1.5">Team – Group 11</span>
    <br/>
    <span style = "font-family:Arial; font-size:11pt;line-height: 1.5">Braxton Ratekin</span>
    <br/>
    <span style = "font-family:Arial; font-size:11pt;line-height: 1.5">Carmen Galgano</span>
    <br/>
    <span style = "font-family:Arial; font-size:11pt;line-height: 1.5">Anand Kumar Babu</span>
</center>

<span style = "font-family:Arial; font-size:11pt;line-height: 1.5;">
Using the GoodReads dataset from Kaggle (<a href = "https://www.kaggle.com/datasets/thedevastator/comprehensive-overview-of-52478-goodreads-best-b">View Dataset</a>), the project attempts to provide an online Library Management and Books Recommender system (LMBRS) to manage the books maintenance, borrowing activities, and to provide personalized reading experience through recommendation based on user preferences, reading history and other relevant factors.
The integrated library management and Books recommender system will provide not only the library management insights on the types of books most preferred by larger group of people but also provide personalized recommendation to users based on their preferences.
Besides, there are opportunities to expand the business functions of the system beyond the scope mentioned above (Ref: Project Usefulness).
</span>

<hr style = "color:darkgray;height:3px;">

<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">About the Notebook</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">

This notebooks performs the following activities in order to prepare data and to create database for the LMBRS system.
    <ul>
        <li>Study the <a href = "https://www.kaggle.com/datasets/thedevastator/comprehensive-overview-of-52478-goodreads-best-b">GoodReads dataset</a></li>
        <li>Cleanse the dataset to eliminate non-ascii data</li>
        <li>Identify redundant data in the dataset and attempt to normalize</li>
        <li>Format data to the appropriate data types</li>
        <li>Connect MySQL DBMS, create a schema called LMBRS</li>
        <li>Read the SQL script from the file system and execute commands to create the tables required</li>
        <li>Copy the normalized data from the GoodReads dataset to the corresponding tables</li>
        <li>Insert initial set of data in to certain tables</li>
        <li>Perform some meaningful sample queries on the database</li>
    </ul>
</span>
<span style = "font-family:Arial;font-size:10pt;line-height:1.5;color:red;font-style:italic;">Note: Dataset download is perfomed outside of this notebook</span><br/><br/>
<hr style = "color:darkgray;height:3px;">

<span style = "font-family:Arial; font-size:16pt; font-weight:bold;color:purple;">Imports and Functions</span>

<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">Imports</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">All libraries / packages requires for this notebook</span>

In [1]:
import pandas as pd
import sys
from datetime import datetime
import time
import random
import sqlite3
import os


from IPython.core.display import HTML

<br/><span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">Functions</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">All utility functions that are used for various operations in the notebook</span>

In [2]:
def get_unique_values(df, column, is_csv = True, is_array = False):
    unique_items = {'':1}
    i = 2
    for ser_index in df[column].value_counts().index:
        ser_index = strip_str(ser_index)
        for item in split_CSV(ser_index, is_csv, is_array):
            item = strip_char_at_ends(strip_char_at_ends(item, "\""), "'").strip().title()
            if not item in unique_items:
                unique_items[item] = i
                i += 1
    return unique_items


def split_CSV(data, is_csv = True, is_array = False):
    data = str(data)
    if is_array: data = arr_str_to_csv(data)
    if is_csv: data = data.split(",")
    else: data = [data]
    return data
    

def arr_str_to_csv(data):
    data = data.strip()
    start, end = data.find("["), data.find("]")
    if start == 0 and end == len(data) - 1 and end > start:
        data = data[1:end]
    return data.strip()


def strip_char_at_ends(data, char):
    data = data.strip()
    start = data.find(char)
    if start == 0:
        end = data.rfind(char)
        if end == len(data) - 1:
            data = data[1:end]
    return data.strip()


def strip_str(data):
    return strip_brace(data.strip().replace("-", " ").replace("_", " "))


def strip_brace(data):
    start, end = data.find("("), data.find(")")
    while start != -1 and end != -1 and start < end:
        data = data[0:start] + data[end + 1:]
        start, end = data.find("("), data.find(")")
    return data.strip()


def create_book_id_link(book_id, data, columns, data_dict, ref_df, is_csv = True, is_array = False):
    data = strip_str(data)
    for item in split_CSV(data, is_csv, is_array):
        item = strip_char_at_ends(strip_char_at_ends(item, "\""), "'").strip().title()
        if item in data_dict.keys():
            ref_df = ref_df.append(pd.Series([book_id, data_dict[item]], index = ref_df.columns), ignore_index = True)
        else:
            print("Not Found ==> ", book_id, ":", item, ":", columns)
    return ref_df


def create_df(columns, data_dict):
    df = pd.DataFrame(columns = columns)
    df[columns[0]], df[columns[1]] = list(data_dict.values()), list(data_dict.keys())
    return df


def print_table_info(msg, df):
    print(msg, ":", df.shape)
    display(df.head())
    print("\n\n")
    
    
def rreplace(data, str_find, str_replace):
    return (data[::-1].replace(str_find[::-1], str_replace[::-1], 1))[::-1]


def format_date(date_str):
    date_str = date_str.replace("Romantic Releases", "").replace("Expected publication:", "")
    date_str = rreplace(rreplace(rreplace(rreplace(date_str, "st", ""), "th", ""), "rd", ""), "nd", "").strip()
    date_str = date_str.replace("Augu ", "August ")
    #print(date_str)
    try:
        date_str = str(datetime.strptime(date_str, "%m/%d/%y").date())
    except:
        try:
            date_str = str(datetime.strptime(date_str, "%B %d %Y").date())
        except:
            try:
                date_str = str(datetime.strptime(date_str, "%B %d %y").date())
            except:
                try:
                    date_str = str(datetime.strptime(date_str, "%B %Y").date())
                except:
                    try:
                        date_str = str(datetime.strptime(date_str, "%Y").date())
                    except:
                        date_str = "01/01/01"
    return date_str


def format_price(price):
    price_str = str(price)
    indx2 = price_str.replace(".", "#", 1).find(".")
    if indx2 > 0: price_str = price_str[0:indx2]
    return float(price_str)


def copy_df_to_table(df, columns, table_name):
    if columns is None:
        df.to_sql(table_name, con = engine, if_exists = 'append', index = False, chunksize = 1000)
    else:
        df[columns].to_sql(table_name, con = engine, if_exists = 'append', index = False, chunksize = 1000)
    

def copy_df_to_table_sqlite(df, columns, table_name):
    if columns is None:
        df.to_sql(name = table_name, con = mydb, if_exists = 'append', index = False, chunksize = 1000)
    else:
        df[columns].to_sql(name = table_name, con = mydb, if_exists = 'append', index = False, chunksize = 1000)


def truncate_text(text, length):
    if len(text) > length: return text[0:length]
    else: return text


def display_message(msg, is_code = False, header = None):
    html = "<br/><table width = '100%' cellspacing = 0 cellpadding = 0 style = 'border:1px solid darkgray'>"
    font = "Courier" if is_code else "Arial"
    if not header is None and len(header.strip()) > 0:
        html += "<tr style = 'background-color:darkgreen'><th style = 'text-align:left;font-family:Arial;font-size:11pt;color:white;font-weight:bold'>" + header.strip() + "</th>"
    for item in msg:
        item = item.strip()
        if len(item) > 0:
            html += "<tr><td style = 'text-align:left;font-family:" + font + ";font-size:11pt;background-color:#EEEEEE;border:1px solid lightgray;'>&#9758;&nbsp;&nbsp;" + item + "</td></tr>"
    html += "</table><br/>"
    display(HTML(html))


def display_SQL_messages(sql):
    header = None
    msg = []
    for item in sql:
        if item.startswith("#"):
            if len(msg) > 0: 
                display_message(msg, is_code = True, header = header)
                header = None
                msg = []
            header = item.strip("#")
        else:
            msg.append(item.strip())
    if len(msg) > 0: display_message(msg, is_code = True, header = header)

        
def exec_query(c, title, query, display_as_df = True):
    c.execute(query)
    if display_as_df:
        colnames = c.description
        columns = []
        display(HTML("<br/><b style = 'color:blue;text-decoration:underline'>" + title + "</b>"))
        for row in colnames: columns.append(row[0])
        df = pd.DataFrame(c.fetchall(), columns=columns)
        display(df)
        display(HTML("<br/>"))
    else:
        for row in rs:
            print(row)

<br/><hr style = "color:darkgray;height:3px;">

In [3]:
if os.path.isfile("lmbrs.db"):
    display_message(["<b style = 'color:red'>The DB file already exists in the folder. Please delete this file and rerun the notebook. Exiting...</b>"])
    sys.exit(0)

<span style = "font-family:Arial; font-size:16pt; font-weight:bold;color:purple;">Data Ingestion</span>
<ul>
        <li>Read data from the donloaded file</li>
        <li>View dataset shape</li>
        <li>Study the features</li>
        <li>View data type of the features</li>
</ul>

<br/>
<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">About Dataset</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Read data from file and study the dataset characteristics</span>

In [4]:
books = pd.read_csv("books_1.Best_Books_Ever.csv")

In [5]:
display_message([
    "The dataset has " + str(books.shape[0]) + " rows and " + str(books.shape[1]) + " columns.",
    "Most of the fields are object types. Looking at the data, they mostly are strings.",
    "The columns that may not be required for the application can be dropped.",
], header = "Observations")

Observations
☞ The dataset has 52478 rows and 25 columns.
"☞ Most of the fields are object types. Looking at the data, they mostly are strings."
☞ The columns that may not be required for the application can be dropped.


In [6]:
books.drop(columns=["bookId", "characters", "firstPublishDate", "ratingsByStars", "setting", "bbeScore", "bbeVotes"], inplace=True)

<br/>
<hr style = "color:darkgray;height:3px;">

<br/>
<span style = "font-family:Arial; font-size:16pt; font-weight:bold;color:purple;">Data Cleansing</span>
<ul>
        <li>Dataset has multiple columns with non-ascii characters</li>
        <li>This application will be dealing with ascii charset</li>
        <li>Drop all the rows where there are non-ascii data in the text columns</li>
        <li>View data type of the features</li>
</ul>

<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">Drop Non-ascii data</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Drop rows that contain non-ascii characters in any of their text columns</span>

In [7]:
start_time = time.time()
# Drop rows with non-ascii characters in any of the columns
rows_dropped = 0
for row_idx, row in books.iterrows(): 
    for _, col in row.iteritems():
        if not str(col).isascii():
            books.drop([row_idx], inplace = True)
            rows_dropped += 1
            break

print("--- Completed in %s seconds ---" % (time.time() - start_time))
display_message([
                    "Total Number of non-ascii rows dropped : " + str(rows_dropped),
                    "New size of the dataset after dropping non-ascii rows is " + str(books.shape[0]) + " rows x " + str(books.shape[1]) + " columns."
                ])

--- Completed in 217.57125210762024 seconds ---


☞ Total Number of non-ascii rows dropped : 25989
☞ New size of the dataset after dropping non-ascii rows is 26489 rows x 18 columns.


<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">Replace 'NA'</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Replace null values appropriately</span>

In [8]:
na_count = books.isna().sum()
display_message([str(idx) + " : " + str(na_count[i]) for i, idx in enumerate(na_count.index)], header = "Count of null values by column")

Count of null values by column
☞ title : 0
☞ series : 14829
☞ author : 0
☞ rating : 0
☞ description : 825
☞ language : 2095
☞ isbn : 0
☞ genres : 0
☞ bookFormat : 461
☞ edition : 24363


Since the dataset is being prepared for transactional purpose, there is no need to drop or do any additional calculation to fill null values. They can be replaced with a <i>&lt;BLANK&gt;</i> in columns containing string values and -1 in columns with numeric values. Here '-1' is used instead of zero so that the application can identify that this is column of the row had no data.

In [9]:
num_cols = {'rating', 'pages', 'numRatings', 'likedPercent', 'price'}
for col in books.columns:
    replace = -1 if col in num_cols else ''
    books[col].fillna(replace, inplace=True)

In [10]:
display_message([
                    "The dataset was found to have columns with non-ascii strings. These rows were dropped.",
                    "Also, there were multiple columns with null values.",
                    "The null values in columns with string data are replaced with <i>&lt;BLANK&gt;</i>",
                    "And the null values in columns with numeric data are replaced with -1"
                ], header = "Observations")

Observations
☞ The dataset was found to have columns with non-ascii strings. These rows were dropped.
"☞ Also, there were multiple columns with null values."
☞ The null values in columns with string data are replaced with <BLANK>
☞ And the null values in columns with numeric data are replaced with -1


<hr style = "color:darkgray;height:3px;">
<br/>

<span style = "font-family:Arial; font-size:16pt; font-weight:bold;color:purple;">Data Normalization</span>
<ul>
        <li>Normalize the redundant data in the dataset</li>
        <li>The following features are found to contain values that are redundant. These are good candidates for normilization.
            <ul>
                <li><i>Author</i></li>
                <li><i>Language</i></li>
                <li><i>Genre</i></li>
                <li><i>Book Format</i></li>
                <li><i>Publisher</i></li>
                <li><i>Award</i></li>
            </ul>
    </li>
</ul>

<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">Identify Unique Values</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Parse, Sanitize and Store Unique values in each of the above-mentioned columns with redundant data</span>

In [11]:
#Added by Anand for Cloud DB. Pick only 10000 rows from the dataframe
books = books.sample(n = 10000)
books.shape

(10000, 18)

In [12]:
unique_authors = get_unique_values(books, "author", is_csv = True, is_array = False)
unique_languages = get_unique_values(books, "language", is_csv = False, is_array = False)
unique_genres = get_unique_values(books, "genres", is_csv = True, is_array = True)
unique_book_formats = get_unique_values(books, "bookFormat", is_csv = True, is_array = False)
unique_publishers = get_unique_values(books, "publisher", is_csv = False, is_array = False)
unique_awards = get_unique_values(books, "awards", is_csv = True, is_array = True)

In [13]:
display_message([
                    "Authors : " + str(len(unique_authors.keys())),
                    "Languages : " + str(len(unique_languages.keys())),
                    "Genres : " + str(len(unique_genres.keys())),
                    "Book Formats : " + str(len(unique_book_formats.keys())),
                    "Publishers : " + str(len(unique_publishers.keys())),
                    "Awards : " + str(len(unique_awards.keys())),
                ], header = "Unique Values")

Unique Values
☞ Authors : 8438
☞ Languages : 28
☞ Genres : 800
☞ Book Formats : 45
☞ Publishers : 2964
☞ Awards : 1624


<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Add an 'id' column to the dataframe with values from 1 to number of rows in the df to uniquely identify each book. This field will be used later for merging other dataframes.</span>

In [14]:
books.insert(loc = 0, column = 'id', value = range(1, books.shape[0] + 1))

In [15]:
books.head(5)

,id,title,series,author,rating,description,language,isbn,genres,bookFormat,edition,pages,publisher,publishDate,awards,numRatings,likedPercent,coverImg,price
36762,1,Rider,Spirals of Destiny #1,Jim Bernheimer (Goodreads Author),3.85,Forget everything you thought you knew about u...,English,9780982508763,"['Fantasy', 'Audiobook', 'Fiction', 'Unicorns'...",Paperback,,250,Gryphonwood Press,June 10th 2010,[],335,92.0,https://i.gr-assets.com/images/S/compressed.ph...,5.20
38589,2,Amy Falls Down,Amy Gallup #2,Jincy Willett,3.67,Amy Gallup is an aging novelist and writing in...,,9781250028280,"['Fiction', 'Humor', 'Contemporary', 'Adult', ...",ebook,,336,Thomas Dunne Books,July 9th 2013,[],2222,88.0,https://i.gr-assets.com/images/S/compressed.ph...,-1
17822,3,The Everything Enneagram Book: Identify Your T...,,Susan Reynolds (Goodreads Author),3.71,Each of us wears our personality like a disgui...,English,9781598692761,"['Nonfiction', 'Psychology', 'Self Help', 'Rel...",Paperback,,320,Everything,July 17th 2007,[],124,89.0,https://i.gr-assets.com/images/S/compressed.ph...,5.40
17643,4,Selected Poems,,"Thomas Hardy, Robert Mezey (Editor)",3.97,,English,9780140436990,"['Poetry', 'Classics', 'British Literature', '...",Paperback,Penguin Classics,320,Penguin Books,December 1st 1998,[],1794,94.0,https://i.gr-assets.com/images/S/compressed.ph...,3.69
37920,5,Horrid Henry and the Soccer Fiend,Horrid Henry #14,"Francesca Simon, Tony Ross (Illustrator)",4.08,Age Level: 7 and up - Grade Level: 2 to 5Horri...,English,9781402217784,"['Childrens', 'Fiction', 'Humor', 'Middle Grad...",Paperback,,101,Sourcebooks Jabberwocky,July 1st 2009,[],726,92.0,https://i.gr-assets.com/images/S/compressed.ph...,1.45


<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Create Dataframes that links each book id with its corresponding 
    <ul>
        <li>author ids (1 book Id to Many author Id)</li>
        <li>language id (1 book id to 1 language id)</li>
        <li>genre ids (1 book id to many genre id)</li>
        <li>format id (1 book id to many format id)</li>
        <li>publisher id (1 book id to 1 publisher id)</li>
        <li>award id (1 book id to many award id)</li>
    </ul>
</span>

In [16]:
start_time = time.time()

references = [
    {"field": "author", "source_df": unique_authors, "target_df": pd.DataFrame(columns = ["book_id", "author_id"]), "is_csv": True, "is_array": True},
    {"field": "language", "source_df": unique_languages, "target_df": pd.DataFrame(columns = ["book_id", "lang_id"]), "is_csv": False, "is_array": False},
    {"field": "genres", "source_df": unique_genres, "target_df": pd.DataFrame(columns = ["book_id", "genre_id"]), "is_csv": True, "is_array": True},
    {"field": "bookFormat", "source_df": unique_book_formats, "target_df": pd.DataFrame(columns = ["book_id", "format_id"]), "is_csv": True, "is_array": False},
    {"field": "publisher", "source_df": unique_publishers, "target_df": pd.DataFrame(columns = ["book_id", "publisher_id"]), "is_csv": False, "is_array": False},
    {"field": "awards", "source_df": unique_awards, "target_df": pd.DataFrame(columns = ["book_id", "award_id"]), "is_csv": True, "is_array": True}
]


for row_idx, row in books.iterrows():
    for item in references:
        item["target_df"] = create_book_id_link(row["id"], row[item["field"]], item["target_df"].columns, item["source_df"], item["target_df"], item["is_csv"], item["is_array"])
    if row["id"] % 1000 == 0: print("Processed :", row["id"])

books_authors, books_lang, books_genres, books_formats, books_publishers, books_awards = references[0]["target_df"], references[1]["target_df"], references[2]["target_df"], references[3]["target_df"], references[4]["target_df"], references[5]["target_df"]

print("--- Completed in %s seconds ---" % (time.time() - start_time))

display_message([
    "Authors : " + str(books_authors.shape[0]) + " x " + str(books_authors.shape[1]),
    "Languages : " + str(books_lang.shape[0])  + " x " +  str(books_lang.shape[1]),
    "Genres : " + str(books_genres.shape[0]) + " x " +  str(books_genres.shape[1]),
    "Formats : " + str(books_formats.shape[0])  + " x " +  str(books_formats.shape[1]),
    "Publishers : " + str(books_publishers.shape[0])  + " x " +  str(books_publishers.shape[1]),
    "Awards : " + str(books_awards.shape[0])  + " x " +  str(books_awards.shape[1]),
], header = "Books - Other Entities Relationships")

Processed : 1000
Processed : 2000
Processed : 3000
Processed : 4000
Processed : 5000
Processed : 6000
Processed : 7000
Processed : 8000
Processed : 9000
Processed : 10000
--- Completed in 86.31511807441711 seconds ---


Books - Other Entities Relationships
☞ Authors : 12618 x 2
☞ Languages : 10000 x 2
☞ Genres : 78266 x 2
☞ Formats : 10002 x 2
☞ Publishers : 10000 x 2
☞ Awards : 11703 x 2


In [17]:
display_message(
    [
        "Unique Authors : " + str(len(unique_authors.keys())),
        "Unique Languages : " + str(len(unique_languages.keys())),
        "Unique Genres : " + str(len(unique_genres.keys())),
        "Unique Formats : " + str(len(unique_book_formats.keys())),
        "Unique Publishers : " + str(len(unique_publishers.keys())),
        "Unique Awards : " + str(len(unique_awards.keys())),
    ], header = "Number of Unique Entities "
)

Number of Unique Entities
☞ Unique Authors : 8438
☞ Unique Languages : 28
☞ Unique Genres : 800
☞ Unique Formats : 45
☞ Unique Publishers : 2964
☞ Unique Awards : 1624


<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Since, the unique entities are identified and stored in a separate dataframe, the columns in the 'books' dataframe can be dropped.</span>

In [18]:
books.drop(columns = ["author", "language", "genres", "bookFormat", "publisher", "awards"], inplace = True)

<br/>
<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">Merge Ids to books Dataframe</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Parse, Sanitize and Store Unique values in each of the above-mentioned columns with redundant data</span>

<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Since, each book id is associated with one language and one publisher, the language id and publisher id can be merged to the same database.</span>

In [19]:
books = pd.merge(books, books_lang, left_on='id', right_on='book_id')

In [20]:
books.drop(columns = ["book_id"], inplace = True)

In [21]:
books = pd.merge(books, books_publishers, left_on='id', right_on='book_id')

In [22]:
books.drop(columns = ["book_id"], inplace = True)

<br/>
<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">Book Id - Entities Id Mapping Dataframes</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Create Dataframes with book ids mapped to entities id. This would help copy the data to the DB tables.</span>

In [23]:
authors_df = create_df(["id", "name"], unique_authors)
languages_df = create_df(["id", "name"], unique_languages)
genres_df = create_df(["id", "type"], unique_genres)
formats_df = create_df(["id", "type"], unique_book_formats)
publishers_df = create_df(["id", "name"], unique_publishers)
awards_df = create_df(["id", "name"], unique_awards)

In [24]:
# All tables generated out of raw data

all_tables = {"Books Master":books, "Authors Master":authors_df, "Language Master":languages_df, 
              "Genre Master":genres_df, "Book Format Master":formats_df, 
              "Publishers Master": publishers_df, "Awards Master": awards_df,
              "Books - Authors Relationship":books_authors, "Books - Genres Relationship": books_genres,
              "Books - Formats Relationship":books_formats, "Books - Awards Relationship": books_awards
             }

for key, value in all_tables.items(): print_table_info(key, value)

Books Master : (10000, 15)


,id,title,series,rating,description,isbn,edition,pages,publishDate,numRatings,likedPercent,coverImg,price,lang_id,publisher_id
0,1,Rider,Spirals of Destiny #1,3.85,Forget everything you thought you knew about u...,9780982508763,,250,June 10th 2010,335,92.0,https://i.gr-assets.com/images/S/compressed.ph...,5.20,2,1053
1,2,Amy Falls Down,Amy Gallup #2,3.67,Amy Gallup is an aging novelist and writing in...,9781250028280,,336,July 9th 2013,2222,88.0,https://i.gr-assets.com/images/S/compressed.ph...,-1,1,372
2,3,The Everything Enneagram Book: Identify Your T...,,3.71,Each of us wears our personality like a disgui...,9781598692761,,320,July 17th 2007,124,89.0,https://i.gr-assets.com/images/S/compressed.ph...,5.40,2,2275
3,4,Selected Poems,,3.97,,9780140436990,Penguin Classics,320,December 1st 1998,1794,94.0,https://i.gr-assets.com/images/S/compressed.ph...,3.69,2,4
4,5,Horrid Henry and the Soccer Fiend,Horrid Henry #14,4.08,Age Level: 7 and up - Grade Level: 2 to 5Horri...,9781402217784,,101,July 1st 2009,726,92.0,https://i.gr-assets.com/images/S/compressed.ph...,1.45,2,615





Authors Master : (8438, 2)


,id,name
0,1,
1,2,Stephen King
2,3,Lucian Bane
3,4,Nora Roberts
4,5,Agatha Christie





Language Master : (28, 2)


...

,book_id,genre_id
0,1,3
1,1,40
2,1,2
3,1,421
4,1,46





Books - Formats Relationship : (10002, 2)


,book_id,format_id
0,1,2
1,2,6
2,3,2
3,4,2
4,5,2





Books - Awards Relationship : (11703, 2)


,book_id,award_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


<hr style = "color:darkgray;height:3px;">
<br/>

<span style = "font-family:Arial; font-size:16pt; font-weight:bold;color:purple;">Data Formatting</span>
<ul>
        <li>Cast data to appropriate data types</li>
        <li>Replace inconsistent data with the correct data</li>
</ul>

<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Replace the data '1 page' in the column pages with 1. This column is expected to hold numeric data.</span>

In [25]:
books.pages[books.pages == "1 page"] = 1

/var/folders/6v/4q9b5flx2l930cf571hlj8q00000gn/T/ipykernel_69976/1021115103.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books.pages[books.pages == "1 page"] = 1


<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Format various formats of the date to a consistent format. When there are no date values replace with '01/01/01'</span>

In [26]:
books["publishDate"] = books["publishDate"].apply(lambda x: format_date(x))

<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Format price values. Some values of the format 1.2236.767.989. These are formatted to appropriate decimal values.</span>

In [27]:
books["price"] = books["price"].apply(lambda x: format_price(x))

<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Format all columns of the dataframe to required data types</span>

In [28]:
books = books.astype({"id":int, "title":str, "series":str, "rating":float, "description":str, "isbn":str,
                        "edition":str, "pages":int, "publishDate":str, "numRatings":int, "likedPercent":float,
                        "coverImg":str, "price":float, "lang_id":int, "publisher_id":int})

In [29]:
books.head()

,id,title,series,rating,description,isbn,edition,pages,publishDate,numRatings,likedPercent,coverImg,price,lang_id,publisher_id
0,1,Rider,Spirals of Destiny #1,3.85,Forget everything you thought you knew about u...,9780982508763,,250,2010-06-10,335,92.0,https://i.gr-assets.com/images/S/compressed.ph...,5.20,2,1053
1,2,Amy Falls Down,Amy Gallup #2,3.67,Amy Gallup is an aging novelist and writing in...,9781250028280,,336,2013-07-09,2222,88.0,https://i.gr-assets.com/images/S/compressed.ph...,-1.00,1,372
2,3,The Everything Enneagram Book: Identify Your T...,,3.71,Each of us wears our personality like a disgui...,9781598692761,,320,2007-07-17,124,89.0,https://i.gr-assets.com/images/S/compressed.ph...,5.40,2,2275
3,4,Selected Poems,,3.97,,9780140436990,Penguin Classics,320,1998-12-01,1794,94.0,https://i.gr-assets.com/images/S/compressed.ph...,3.69,2,4
4,5,Horrid Henry and the Soccer Fiend,Horrid Henry #14,4.08,Age Level: 7 and up - Grade Level: 2 to 5Horri...,9781402217784,,101,2009-07-01,726,92.0,https://i.gr-assets.com/images/S/compressed.ph...,1.45,2,615


<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Create 1 copy for each book</span>

In [30]:
ids = list(books["id"])
borrow_status_id = [1 for i in range(0, len(ids))]
book_copy = pd.DataFrame(columns = ["book_id", "borrow_status_id"])
book_copy["book_id"], book_copy["borrow_status_id"] = ids, borrow_status_id

<hr style = "color:darkgray;height:3px;">
<br/>

<span style = "font-family:Arial; font-size:16pt; font-weight:bold;color:purple;">Database Operations</span>
<ul>
        <li>Connect to MySQL</li>
        <li>Create schema and tables</li>
        <li>Insert data in to the tables created</li>
</ul>

<br/>
<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">SQL Commands</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Read the SQL file and split in to individual commands</span>

In [78]:
f = open("Create_Tables_SQLite.sql")
sql = ""
for line in f.readlines():
    sql += line.strip()

f.close()

sql = sql.split(";")

<br/>
<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">Create DB Objects</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Connect to MySQL and create schema and tables</span>

In [79]:
mydb = sqlite3.connect('lmbrs.db')
cursor = mydb.cursor()

In [80]:
for item in sql:
    cursor.execute(item)
mydb.commit()

display_SQL_messages(sql)

"☞ CREATE TABLE `person_status` (`id` INTEGER PRIMARY KEY AUTOINCREMENT,`status` varchar(100) NOT NULL)"
"☞ CREATE TABLE `person` (`id` INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,`f_name` varchar(50) DEFAULT NULL,`l_name` varchar(50) DEFAULT NULL,`email` varchar(200) DEFAULT NULL,`logon_id` varchar(30) NOT NULL UNIQUE,`password` varchar(50) DEFAULT NULL,`last_logged_on` datetime DEFAULT NULL,`is_admin` tinyint(1) DEFAULT '0',`status_id` int DEFAULT NULL,`created_by` int DEFAULT NULL,`created_on` datetime DEFAULT NULL,CONSTRAINT `person_person_status_id` FOREIGN KEY (`status_id`) REFERENCES `person_status` (`id`) ON UPDATE CASCADE)"
"☞ CREATE TABLE `publisher` (`id` INTEGER PRIMARY KEY AUTOINCREMENT,`name` varchar(350) NOT NULL)"
"☞ CREATE TABLE `language` (`id` INTEGER PRIMARY KEY AUTOINCREMENT,`name` varchar(100) NOT NULL)"
"☞ CREATE TABLE `genre` (`id` INTEGER PRIMARY KEY AUTOINCREMENT,`type` varchar(100) NOT NULL)"
"☞ CREATE TABLE `format` (`id` INTEGER PRIMARY KEY AUTOINCREMENT,`type` varchar(150) NOT NULL)"
"☞ CREATE TABLE `author` (`id` INTEGER PRIMARY KEY AUTOINCREMENT,`name` varchar(100) NOT NULL)"
"☞ CREATE TABLE `award` (`id` INTEGER PRIMARY KEY AUTOINCREMENT,`name` varchar(350) NOT NULL)"
"☞ CREATE TABLE `book` (`id` INTEGER PRIMARY KEY AUTOINCREMENT,`title` varchar(350) NOT NULL,`series` varchar(250) DEFAULT NULL,`rating` decimal(5,2) DEFAULT '-1.00',`description` varchar(2500) DEFAULT NULL,`isbn` varchar(30) DEFAULT NULL,`edition` varchar(150) DEFAULT NULL,`pages` int(10) DEFAULT '-1',`publishDate` date DEFAULT NULL,`numRatings` int(11) DEFAULT NULL,`likedPercent` decimal(5,2) DEFAULT NULL,`coverImg` varchar(300) DEFAULT NULL,`price` decimal(5,2) DEFAULT '-1.00',`lang_id` int DEFAULT NULL,`publisher_id` int DEFAULT NULL,CONSTRAINT `book_lang_id` FOREIGN KEY (`lang_id`) REFERENCES `language` (`id`) ON DELETE CASCADE ON UPDATE CASCADE,CONSTRAINT `book_publisher_id` FOREIGN KEY (`publisher_id`) REFERENCES `publisher` (`id`) ON DELETE CASCADE ON UPDATE CASCADE)"
"☞ CREATE TABLE `book_author` (`id` INTEGER PRIMARY KEY AUTOINCREMENT,`book_id` int NOT NULL,`author_id` int NOT NULL,CONSTRAINT `book_author_author_id` FOREIGN KEY (`author_id`) REFERENCES `author` (`id`) ON DELETE CASCADE ON UPDATE CASCADE,CONSTRAINT `book_author_book_id` FOREIGN KEY (`book_id`) REFERENCES `book` (`id`) ON DELETE CASCADE ON UPDATE CASCADE)"
"☞ CREATE TABLE `book_award` (`id` INTEGER PRIMARY KEY AUTOINCREMENT,`book_id` int NOT NULL,`award_id` int NOT NULL,CONSTRAINT `book_award_award_id` FOREIGN KEY (`award_id`) REFERENCES `award` (`id`) ON DELETE CASCADE ON UPDATE CASCADE,CONSTRAINT `book_award_book_id` FOREIGN KEY (`book_id`) REFERENCES `book` (`id`) ON DELETE CASCADE ON UPDATE CASCADE)"


<br/>
<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">Data Presets</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Insert preset data in to tables. Create persons and add random borrowals in to the borrowal table.</span>

In [81]:
f = open("Data_Preset_SQLite_1.sql")
sql = ""
for line in f.readlines():
    sql += line.strip()

f.close()

sql = sql.split(";")

In [82]:
# Data Preset for the application
for item in sql:
    cursor.execute(item)
mydb.commit()

display_SQL_messages(sql)

"☞ INSERT INTO person_status(status) VALUES ('Active'), ('Suspended'), ('Deleted')"
"☞ INSERT INTO borrow_status(status) VALUES ('Available'), ('Borrow Blocked'), ('Borrowed'), ('Return Waiting'), ('Returned')"
"☞ INSERT INTO person(f_name, l_name, email, logon_id, password, last_logged_on, is_admin, status_id, created_by, created_on)VALUES ('Anand kumar', 'Babu', 'anbabu@iu.edu', 'anbabu', 'b''YW1vb25ndXNz''', '2023-07-08', 1, 1, 1, CURRENT_TIMESTAMP),('Carmen', 'Galgano', 'cgalgano@iu.edu', 'cgalgano', 'b''YXJhcXVhbmlk''', '2023-07-08', 0, 1, 1, CURRENT_TIMESTAMP),('Braxton Thatcher', 'Ratekin', 'btrateki@iupui.edu', 'btrateki', 'b''YmVoZWV5ZW0=''', '2023-07-08', 1, 1, 1, CURRENT_TIMESTAMP),('Rachel', 'Booker', 'rachel@unknown.com', 'booker12', 'b''YmxhY2VwaGFsb24=''', '2023-07-08', 0, 1, 1, CURRENT_TIMESTAMP),('Laura', 'Grey', 'laura@unknown.com', 'grey07', 'b''Y2VsZXN0ZWVsYQ==''', '2023-07-08', 0, 1, 1, CURRENT_TIMESTAMP),('Craig', 'Johnson', 'craig@unknown.com', 'johnson81', 'b''Y2hhcm1hbmRlcg==''', '2023-07-08', 0, 1, 1, CURRENT_TIMESTAMP),('Mary', 'Jenkins', 'mary@unknown.com', 'jenkins46', 'b''ZHJhZ2FwdWx0''', '2023-07-08', 0, 1, 1, CURRENT_TIMESTAMP),('Jamie', 'Smith', 'jamie@unknown.com', 'smith79', 'b''ZHVuc3BhcmNl''', '2023-07-08', 0, 1, 1, CURRENT_TIMESTAMP)"


In [83]:
cursor.close()

<br/>
<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">Copy data to tables</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Copy data from Dataframes to appropriate tables</span>

<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Trim text information to fit the defined data length in the tables</span>

In [84]:
df_names = {"books": books, "authors_df": authors_df, "languages_df": languages_df, "genres_df": genres_df, "formats_df": formats_df, 
            "publishers_df": publishers_df, "awards_df": awards_df}

column_size = {
                "books": {"title":350, "series":250, "description":2500, "isbn":30, "edition":150},
                "authors_df": {"name": 100}, "languages_df": {"name": 100}, "genres_df": {"type": 100},
                "formats_df": {"type": 150}, "publishers_df": {"name": 350}, "awards_df": {"name": 350}
            }

for name, size in column_size.items():
    for column, length in size.items():
        df_names[name][column] = df_names[name][column].apply(lambda x: truncate_text(x, length))

<br/>
<span style = "font-family:Arial; font-size:13pt;line-height:1.5;color:blue;font-weight:bold;">Insert data</span>
<br/>
<span style = "font-family:Arial; font-size:11pt;line-height:1.5">Insert data from dataframes to corresponding tables</span>

In [85]:
mydb = sqlite3.connect('lmbrs.db')
cursor = mydb.cursor()

In [86]:
table_info = {
        "author": {"fields": ["name"], "df": authors_df},
        "language": {"fields": ["name"], "df": languages_df},
        "genre": {"fields": ["type"], "df": genres_df},
        "format": {"fields": ["type"], "df": formats_df},
        "publisher": {"fields": ["name"], "df": publishers_df},
        "award": {"fields": ["name"], "df": awards_df},
        "book": {"fields": None, "df": books},
        "book_author": {"fields": ["book_id", "author_id"], "df": books_authors},
        "book_format": {"fields": ["book_id", "format_id"], "df": books_formats},
        "book_genre": {"fields": ["book_id", "genre_id"], "df": books_genres},
        "book_award": {"fields": ["book_id", "award_id"], "df": books_awards},
        "book_copy": {"fields": ["book_id", "borrow_status_id"], "df": book_copy}
}

messages = []
for table_name, df_data in table_info.items():
    copy_df_to_table_sqlite(df_data["df"], df_data["fields"], table_name)
    messages.append("Copied " + str(df_data["df"].shape[0]) + " rows to table '" + table_name + "'")

display_message(messages, header = "Copied rows to tables")

Copied rows to tables
☞ Copied 8438 rows to table 'author'
☞ Copied 28 rows to table 'language'
☞ Copied 800 rows to table 'genre'
☞ Copied 45 rows to table 'format'
☞ Copied 2964 rows to table 'publisher'
☞ Copied 1624 rows to table 'award'
☞ Copied 10000 rows to table 'book'
☞ Copied 12618 rows to table 'book_author'
☞ Copied 10002 rows to table 'book_format'
☞ Copied 78266 rows to table 'book_genre'


In [87]:
mydb.commit()
cursor.close()

In [88]:
mydb = sqlite3.connect('lmbrs.db')
cursor = mydb.cursor()

In [89]:
f = open("Data_Preset_SQLite_2.sql")
sql = ""
for line in f.readlines():
    sql += line.strip()

f.close()

sql = sql.split(";")

In [90]:
# Insert Sample borrowal data
for item in sql:
    cursor.execute(item)
mydb.commit()

display_SQL_messages(sql)

"☞ INSERT INTO borrowal (book_copy_id, person_id, borrow_date, return_date) SELECT id, 2, DATE('2023-07-08'), DATE('2023-07-08', '+32 days') from book_copy where id not in (Select book_copy_id from borrowal) order by RANDOM() LIMIT 50"
"☞ INSERT INTO borrowal (book_copy_id, person_id, borrow_date, return_date) SELECT id, 4, DATE('2023-07-20'), DATE('2023-07-20', '+32 days') from book_copy where id not in (Select book_copy_id from borrowal) order by RANDOM() LIMIT 50"
"☞ INSERT INTO borrowal (book_copy_id, person_id, borrow_date, return_date) SELECT id, 5, DATE('2023-07-22'), DATE('2023-07-22', '+32 days') from book_copy where id not in (Select book_copy_id from borrowal) order by RANDOM() LIMIT 50"
"☞ INSERT INTO borrowal (book_copy_id, person_id, borrow_date, return_date) SELECT id, 6, DATE('2023-07-24'), DATE('2023-07-24', '+32 days') from book_copy where id not in (Select book_copy_id from borrowal) order by RANDOM() LIMIT 50"
"☞ INSERT INTO borrowal (book_copy_id, person_id, borrow_date, return_date) SELECT id, 7, DATE('2023-07-26'), DATE('2023-07-26', '+32 days') from book_copy where id not in (Select book_copy_id from borrowal) order by RANDOM() LIMIT 50"
"☞ INSERT INTO borrowal (book_copy_id, person_id, borrow_date, return_date) SELECT id, 8, DATE('2023-07-28'), DATE('2023-07-28', '+32 days') from book_copy where id not in (Select book_copy_id from borrowal) order by RANDOM() LIMIT 50"
☞ UPDATE book_copy SET borrow_status_id = 2 WHERE id IN (SELECT book_copy_id FROM borrowal)


In [91]:
cursor.close()

In [92]:
display_message([
    "LMBRS Schema created",
    "All required tables were created",
    "Preset data required for the application was inserted",
    "Data extracted from the books.csv downloaded from Kaggle was normalized and inserted in to appropriate tables"
], header = "Data Operations")

Data Operations
☞ LMBRS Schema created
☞ All required tables were created
☞ Preset data required for the application was inserted
☞ Data extracted from the books.csv downloaded from Kaggle was normalized and inserted in to appropriate tables


<hr style = "color:darkgray;height:3px;">
<br/>

In [93]:
import base64
data = [
    {"Logon Id": "anbabu", "Password": "amoonguss", "Encoded Pwd": str(base64.b64encode("amoonguss".encode("utf-8"))).replace("'", "''")},
    {"Logon Id": "cgalgano", "Password": "araquanid", "Encoded Pwd": str(base64.b64encode("araquanid".encode("utf-8"))).replace("'", "''")},
    {"Logon Id": "btrateki", "Password": "beheeyem", "Encoded Pwd": str(base64.b64encode("beheeyem".encode("utf-8"))).replace("'", "''")},
    {"Logon Id": "booker12", "Password": "blacephalon", "Encoded Pwd": str(base64.b64encode("blacephalon".encode("utf-8"))).replace("'", "''")},
    {"Logon Id": "grey07", "Password": "celesteela", "Encoded Pwd": str(base64.b64encode("celesteela".encode("utf-8"))).replace("'", "''")},
    {"Logon Id": "johnson81", "Password": "charmander", "Encoded Pwd": str(base64.b64encode("charmander".encode("utf-8"))).replace("'", "''")},
    {"Logon Id": "jenkins46", "Password": "dragapult", "Encoded Pwd": str(base64.b64encode("dragapult".encode("utf-8"))).replace("'", "''")},
    {"Logon Id": "smith79", "Password": "dunsparce", "Encoded Pwd": str(base64.b64encode("dunsparce".encode("utf-8"))).replace("'", "''")}
]

user_df = pd.DataFrame(data)
user_df

,Logon Id,Password,Encoded Pwd
0,anbabu,amoonguss,b''YW1vb25ndXNz''
1,cgalgano,araquanid,b''YXJhcXVhbmlk''
2,btrateki,beheeyem,b''YmVoZWV5ZW0=''
3,booker12,blacephalon,b''YmxhY2VwaGFsb24=''
4,grey07,celesteela,b''Y2VsZXN0ZWVsYQ==''
5,johnson81,charmander,b''Y2hhcm1hbmRlcg==''
6,jenkins46,dragapult,b''ZHJhZ2FwdWx0''
7,smith79,dunsparce,b''ZHVuc3BhcmNl''
